Link: https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/raster-data-processing/crop-raster-data-with-shapefile-in-python/

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from shapely.geometry import mapping
import geopandas as gpd
import rasterio as rio
from rasterio.plot import plotting_extent
from rasterio.mask import mask
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

# Prettier plotting with seaborn
sns.set(font_scale=1.5)

# Get data and set working directory
et.data.get_data("colorado-flood")
os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

In [ ]:
aoi = os.path.join("data", "colorado-flood", "spatial",
                   "boulder-leehill-rd", "clip-extent.shp")

# Open crop extent (your study area extent boundary)
crop_extent = gpd.read_file(aoi)

In [ ]:
# Define relative paths to DTM and DSM files
dtm_path = os.path.join("data", "colorado-flood", "spatial",
                        "boulder-leehill-rd", "pre-flood", "lidar",
                        "pre_DTM.tif")

dsm_path = os.path.join("data", "colorado-flood", "spatial",
                        "boulder-leehill-rd", "pre-flood", "lidar",
                        "pre_DSM.tif")

# Open DTM and DSM files
with rio.open(dtm_path) as src:
    lidar_dtm_im = src.read(1, masked=True)
    spatial_extent = plotting_extent(src)

with rio.open(dsm_path) as src:
    lidar_dsm_im = src.read(1, masked=True)
    spatial_extent = plotting_extent(src)

# Create canopy height model (CHM)    
lidar_chm_im = lidar_dsm_im - lidar_dtm_im

In [ ]:
print('crop extent crs: ', crop_extent.crs)
print('lidar crs: ', lidar_chm.crs)

In [ ]:
# Plot the crop boundary layer
# Note this is just an example so you can see what it looks like
# You don't need to plot this layer in your homework!
fig, ax = plt.subplots(figsize=(6, 6))

crop_extent.plot(ax=ax)

ax.set_title("Shapefile Crop Extent",
             fontsize=16)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ep.plot_bands(lidar_chm_im, cmap='terrain',
              extent=plotting_extent(lidar_chm),
              ax=ax,
              title="Raster Layer with Shapefile Overlayed",
              cbar=False)

crop_extent.plot(ax=ax, alpha=.8)

ax.set_axis_off()

In [ ]:
lidar_chm_path = os.path.join("data", "colorado-flood", "spatial",
                              "boulder-leehill-rd", "outputs", "lidar_chm.tif")

with rio.open(lidar_chm_path) as lidar_chm:
    lidar_chm_crop, lidar_chm_crop_meta = es.crop_image(lidar_chm,crop_extent)

lidar_chm_crop_affine = lidar_chm_crop_meta["transform"]

# Create spatial plotting extent for the cropped layer
lidar_chm_extent = plotting_extent(lidar_chm_crop[0], lidar_chm_crop_affine)

In [ ]:
# Plot your data
ep.plot_bands(lidar_chm_crop[0],
              extent=lidar_chm_extent,
              cmap='Greys',
              title="Cropped Raster Dataset",
              scale=False)
plt.show()

In [ ]:
# Update with the new cropped affine info and the new width and height
lidar_chm_meta.update({'transform': lidar_chm_crop_affine,
                       'height': lidar_chm_crop.shape[1],
                       'width': lidar_chm_crop.shape[2],
                       'nodata': -999.99})
lidar_chm_meta

In [ ]:
# Write data
path_out = os.path.join("data", "colorado-flood", "spatial", 
                        "outputs", "lidar_chm_cropped.tif")

with rio.open(path_out, 'w', **lidar_chm_meta) as ff:
    ff.write(lidar_chm_crop[0], 1)